In [ ]:
import pprint

In [ ]:
# все пути к файлам могут поломаться, ниже есть переменная с путем,
# значение которой можно изменить
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


# Открытие файлов / исправление кодировок / чистка текстов / etc.

In [ ]:
# ипорт всего что только можно импортировать
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import word_tokenize
import os
from os import listdir
from os.path import isfile, join
import re
from collections import Counter
from pprint import pprint
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Путь к папке с текстами "Порнофильмов"

In [ ]:
path_pf = "/content/gdrive/My Drive/kompling/Порнофильмы/"

Список всех альбомов в **albums_pf**

In [ ]:
import os
albums_pf = set()
for top, dirs, files in os.walk(path_pf):
    for nm in files:       
        albums_pf.add(os.path.basename(top))
albums_pf = list(albums_pf)

Список путей ко всем альбомам

In [ ]:
albums_path_pf = set()
for top, dirs, files in os.walk(path_pf):
    for nm in files:       
        albums_path_pf.add(os.path.join(top))
albums_path_pf

{'/content/gdrive/My Drive/kompling/Порнофильмы/2012 Искусство',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2012 На всех экранах страны',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2012 Сколько взорвется бомб',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2012 Скучная Жизнь',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2013 Карма рабочих',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2013 Нищая Страна',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2014 Белые Хлопья',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2014 Молодость и Панк-Рок',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2015 Русская мечта ч1',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2015 Сопротивление',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2016 Как в последний раз',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2016 Русская мечта ч2',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой',
 '/content/gdrive/My Drive/kompling/Порнофиль

In [ ]:
all_songs_path_pf = []
for path in list(albums_path_pf):
  for top, dirs, files in os.walk(path):
      for nm in files:       
          all_songs_path_pf.append(os.path.join(top, nm))
all_songs_path_pf

['/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой/Я так соскучился.txt',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой/Я так боюсь.txt',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой/Только ты.txt',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой/Система.txt',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой/Русский Христос.txt',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой/В диапазоне.txt',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой/Выйди из комнаты.txt',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой/Давай притворимся.txt',
 '/content/gdrive/My Drive/kompling/Порнофильмы/2017 В диапазоне между отчаянием и надеждой/Дл

Исправление разных кодировок

In [ ]:
# получение разных кодировок
def right_enc(path): # ПОЛНЫЙ ПУТЬ С .TXT
  encoding = [
  'utf-8',
  'ANSI',
  'ASCII'
  ]
  # encoding = [
  # 'utf-8',
  # 'cp500',
  # 'utf-16',
  # 'GBK',
  # 'windows-1251',
  # 'ASCII',
  # 'US-ASCII',
  # 'Big5'
  # ]
  correct_encoding = ''
  for enc in encoding:
      try:
          open(path, encoding=enc).read()
      except (UnicodeDecodeError, LookupError):
          pass
      else:
          correct_encoding = enc
          break
  return correct_encoding

In [ ]:
from os import listdir
from os.path import isfile, join
import re

Получение всех файлов в альбоме

In [ ]:
# чистка файлов от мусора
def cleaner(text):
  new = re.sub(r"\[[а-яА-ЯёЁ\d\s\w\.,:;'\"]*\]:", '', text, count=0, flags=0)
  new = re.sub(r"Припев:?", ' ', new, count=0, flags=0)
  new = re.sub(r'((https|http)?://[\S]+)', ' ', new, count=0, flags=0)
  new = re.sub(r'Читать на сайте:', ' ', new, count=0, flags=0)
  new = re.sub(r'Источник:', ' ', new, count=0, flags=0)
  new = re.sub(r'\n', ' ', new, count=0, flags=0)
  new = re.sub(r'…', ' ', new, count=0, flags=0)

  return new

In [ ]:
texts_pf = {}
for album in albums_pf:
  files= [f for f in listdir(path_pf+album) if isfile(join(path_pf+album, f))]
  for elem in files:
    path = path_pf+album+"/"+elem
    with open(path, 'r', encoding='utf-8') as f:
      text = f.read()

      # чистка от мусора в текстах
      new = cleaner(text)

      texts_pf[album] = texts_pf.get(album, '')
      texts_pf[album] += " " + new

In [ ]:
# я не помню что это но боюсь убирать
from nltk.corpus import stopwords
words = [word for word in new if word not in stopwords.words('russian')]

# ФУНКЦИИ ДЛЯ ТОКЕНИЗАЦИИ И ЧАСТОТНИКА

In [ ]:
# ФУНКЦИЯ ДЛЯ ТОКЕНИЗАЦИИ ТЕКСТОВ, РАБОЧАЯ

def tokenize_ru(file_text):
    tokens_t = word_tokenize(file_text)

    tokens_t = [i for i in tokens_t if (i not in string.punctuation)]
    tokens = []
    for i in tokens_t:
      tokens.append(i.lower())

    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'это', 'мой', 'наш', 'твой', 'свой', 'весь', 'по', 'на', 'над', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])
    tokens = [i for i in tokens if (i not in stop_words)]

    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    tokens = [i.replace("'", "").replace("`", "") for i in tokens]
    return tokens
  
def count_frequencies(tokens):

    freq = Counter(tokens).most_common(15)
    print(freq)
    relative = []
    for el in freq:
        word = el[0]
        fr = el[1]/len(tokens)
        relative.append([word, fr])
    
    return relative

def count_frequencies_less(tokens):
    l = len(tokens)
    freq = Counter(tokens).most_common()[:-15:-1]
    print(freq)
    relative = []
    for el in freq:
        word = el[0]
        fr = el[1]/len(tokens)
        relative.append([word, fr])
    
    return relative

def lemmatize(tokens):
    ans = []
    for word in tokens:
      # приведение к начальной форме
      # преобразует слова к неправильным формам
      if word == 'заебись' or word.startswith('ахуе'):
        ans.append(word)
      else:
        ans.append(morph.parse(word)[0][2])

    # да, так делать неправльно, но в слова попадают такие формы как "наш ваш" и так далее, что мешает
    # там, где удаление стоп слов не требуется, используется другая функция
    stop_words = stopwords.words('russian')
    stop_words.extend(['что', 'твой', 'мой', 'это', 'каждый', 'твой', 'свой', 'весь', 'по', 'на', 'над', 'так', 'вот', 'быть', 'как', 'в', '—', '–', 'к', 'на', '...'])
    ans = [i for i in ans if (i not in stop_words)]

    return ans


# Все частотники (по годам и общий)

In [ ]:
lemmas_pf = {}
for album_name, songs in texts_pf.items():
    lemmas_pf[album_name] = lemmatize(tokenize_ru(songs))

In [ ]:
all_lemmas_pf = []
for key, value in lemmas_pf.items():
  for word in value:
    all_lemmas_pf.append(word)

all_lemmas_pf
pprint(count_frequencies(all_lemmas_pf))
pprint(count_frequencies_less(all_lemmas_pf))
# разобраться откуда наш

[('', 92), ('жить', 53), ('наш', 51), ('день', 49), ('стать', 48), ('мир', 48), ('любовь', 47), ('никто', 44), ('страна', 43), ('глаз', 42), ('дом', 41), ('просто', 40), ('хотеть', 39), ('дядя', 39), ('ребёнок', 38)]
[['', 0.01027244305493524],
 ['жить', 0.005917820455560518],
 ['наш', 0.0056945064761054045],
 ['день', 0.00547119249665029],
 ['стать', 0.005359535506922733],
 ['мир', 0.005359535506922733],
 ['любовь', 0.0052478785171951765],
 ['никто', 0.004912907548012505],
 ['страна', 0.004801250558284949],
 ['глаз', 0.004689593568557392],
 ['дом', 0.004577936578829835],
 ['просто', 0.0044662795891022775],
 ['хотеть', 0.004354622599374721],
 ['дядя', 0.004354622599374721],
 ['ребёнок', 0.004242965609647164]]
[('торопливый', 1), ('односторонний', 1), ('припадок', 1), ('генерал', 1), ('восторженно', 1), ('казарма', 1), ('трибунал', 1), ('жир', 1), ('богатый', 1), ('шов', 1), ('трещать', 1), ('привычный', 1), ('ценность', 1), ('улетать', 1)]
[['торопливый', 0.00011165698972755694],
 ['од

In [ ]:
year_sorted_lemmas = {'2012': [],
             '2013': [],
             '2014': [],
             '2015': [],
             '2016': [],
             '2017': [],
             '2019': []}
for key, value in lemmas_pf.items():
  for word in value:
    year_sorted_lemmas[key[:4]].append(word)
  
for key, value in year_sorted_lemmas.items():
  print("Частотник для '" + key + "'")
  pprint(count_frequencies(value))
  print()

Частотник для '2012'
[('', 22), ('жизнь', 18), ('хотеть', 18), ('страна', 13), ('кто-то', 13), ('весна', 12), ('заебись', 10), ('искусство', 9), ('день', 9), ('ждать', 9), ('раб', 9), ('ваш', 8), ('произвести', 8), ('арест', 8), ('поезд', 8)]
[['', 0.012600229095074456],
 ['жизнь', 0.010309278350515464],
 ['хотеть', 0.010309278350515464],
 ['страна', 0.0074455899198167235],
 ['кто-то', 0.0074455899198167235],
 ['весна', 0.006872852233676976],
 ['заебись', 0.0057273768613974796],
 ['искусство', 0.005154639175257732],
 ['день', 0.005154639175257732],
 ['ждать', 0.005154639175257732],
 ['раб', 0.005154639175257732],
 ['ваш', 0.004581901489117984],
 ['произвести', 0.004581901489117984],
 ['арест', 0.004581901489117984],
 ['поезд', 0.004581901489117984]]

Частотник для '2013'
[('ребёнок', 23), ('день', 21), ('никто', 12), ('нужный', 11), ('страна', 9), ('отъебиться', 9), ('мир', 7), ('вор', 7), ('хотеть', 7), ('пойти', 6), ('дом', 6), ('лицо', 6), ('карма', 6), ('рабочий', 6), ('здравствова

# Сквернословие

In [ ]:
def cahnge_to_star(words):
  # замена мата на a*a, т.к. в дальнейшем токенизатор убирает отдельные звездочки
  mat = re.sub(r'\w{0,5}[хx]([хx\s\!@#$%\^&*+-\|\/]{0,6})[уy]([уy\s\!@#$%\^&*+-\|\/]{0,6})[ёiлeеюийя]\w{0,7}|\w{0,6}[пp]([пp\s\!@#$%\^&*+-\|\/]{0,6})[iие]([iие\s\!@#$%\^&*+-\|\/]{0,6})[3зс]([3зс\s\!@#$%\^&*+-\|\/]{0,6})[дd]\w{0,10}|[сcs][уy]([уy\!@#$%\^&*+-\|\/]{0,6})[4чkк]\w{1,3}|\w{0,4}[bб]([bб\s\!@#$%\^&*+-\|\/]{0,6})[lл]([lл\s\!@#$%\^&*+-\|\/]{0,6})[yя]\w{0,10}|\w{0,8}[её][bб][лске@eыиаa][наи@йвл]\w{0,8}|\w{0,4}[еe]([еe\s\!@#$%\^&*+-\|\/]{0,6})[бb]([бb\s\!@#$%\^&*+-\|\/]{0,6})[uу]([uу\s\!@#$%\^&*+-\|\/]{0,6})[н4ч]\w{0,4}|\w{0,4}[еeё]([еeё\s\!@#$%\^&*+-\|\/]{0,6})[бb]([бb\s\!@#$%\^&*+-\|\/]{0,6})[нn]([нn\s\!@#$%\^&*+-\|\/]{0,6})[уy]\w{0,4}|\w{0,4}[еe]([еe\s\!@#$%\^&*+-\|\/]{0,6})[бb]([бb\s\!@#$%\^&*+-\|\/]{0,6})[оoаa@]([оoаa@\s\!@#$%\^&*+-\|\/]{0,6})[тnнt]\w{0,4}|\w{0,10}[ё]([ё\!@#$%\^&*+-\|\/]{0,6})[б]\w{0,6}|\w{0,4}[pп]([pп\s\!@#$%\^&*+-\|\/]{0,6})[иeеi]([иeеi\s\!@#$%\^&*+-\|\/]{0,6})[дd]([дd\s\!@#$%\^&*+-\|\/]{0,6})[oоаa@еeиi]([oоаa@еeиi\s\!@#$%\^&*+-\|\/]{0,6})[рr]\w{0,12}', "a*a", words)
  return mat

def percent_profanity(tokens):
  ans = []
  for el in tokens:
    # регулярка реагирует не на все слова
    if el.startswith('заеби'):
      ans.append(el)
      continue
    for letter in el:
      if letter == "*":
        ans.append("*")
        break

  percent = len(ans) / len(tokens)
  return percent


In [ ]:
# вот та самая нормальная функция
def lemmatize_d(tokens):
    ans = []
    for word in tokens:
      # приведение к начальной форме
      # преобразует слова к неправильным формам
      if word == 'заебись' or word.startswith('ахуе'):
        ans.append(word)
      else:
        ans.append(morph.parse(word)[0][2])

    return ans

In [ ]:
# процент мата от абсолютно всех слов, включая предлоги, местоимения и так далее
years_all_texts = {'2012': '',
             '2013': '',
             '2014': '',
             '2015': '',
             '2016': '',
             '2017': '',
             '2019': ''}
years_profanity = {}
for album, text in texts_pf.items():
  years_all_texts[album[:4]] += text + " "

for year, text in years_all_texts.items():
  percent = percent_profanity(lemmatize_d(tokenize_d(cahnge_to_star(text))))
  years_profanity[year] = percent

years_profanity

{'2012': 0.02101769911504425,
 '2013': 0.01704199634814364,
 '2014': 0.008470798038341507,
 '2015': 0.009685230024213076,
 '2016': 0.0035067212156633548,
 '2017': 0.0024875621890547263,
 '2019': 0.0005733944954128441}

# Части речи

In [ ]:
# токенизация без стоп слов для нормального отображения частей речи
def tokenize_d(file_text):
    tokens_t = word_tokenize(file_text)

    tokens_t = [i for i in tokens_t if (i not in string.punctuation)]
    tokens = []
    for i in tokens_t:
      tokens.append(i.lower())

    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    tokens = [i.replace("'", "").replace("`", "") for i in tokens]
    return tokens

Общие

In [ ]:
def useful_pos(dict_pos):
  readable_pos = {'существительные': 0,
                'глаголы': 0,
                'причастия': 0,
                'деепричастия': 0,
                'прилагательные': 0,
                'наречия': 0}
                # 'местоимения': 0,
                # 'предлоги': 0,
                # 'местоимения': 0,
                # '???': 0,
                # 'другое': 0}
  for pos, num in dict_pos.items():
    if pos == 'NOUN':
      readable_pos['существительные'] += num
    elif pos == 'INFN' or pos == 'VERB':
      readable_pos['глаголы'] += num
    # elif pos == None:
    #   readable_pos['???'] += num
    elif pos == 'ADJF' or pos == 'ADJS' or pos == 'COMP':
      readable_pos['прилагательные'] += num
    elif pos == 'PRTS' or pos == 'PRTF':
      readable_pos['причастия'] += num
    elif pos == 'ADVB':
      readable_pos['наречия'] += num
    elif pos == 'GRND':
      readable_pos['деепричастия'] += num
    # elif pos == 'NPRO':
    #   readable_pos['местоимения'] += num
    # elif pos == 'PREP':
    #   readable_pos['предлоги'] += num
    # else:
    #   readable_pos['другое'] += num

  return readable_pos

In [ ]:
tokens_pf = {}
all_pos_pf = {}
for album_name, songs in texts_pf.items():
    for token in tokenize_d(songs):
      p = morph.parse(token)[0]
      all_pos_pf[p.tag.POS] = all_pos_pf.get(p.tag.POS, 0) + 1
  
useful_pos(all_pos_pf)

{'глаголы': 2467,
 'деепричастия': 76,
 'наречия': 732,
 'прилагательные': 1746,
 'причастия': 101,
 'существительные': 4246}

По годам

In [ ]:
years_pos_pf = {'2012': '',
             '2013': '',
             '2014': '',
             '2015': '',
             '2016': '',
             '2017': '',
             '2019': ''}

# проходим по словарю с текстами по годам
for year, texts in years_all_texts.items():
  year_pos = {}
  for token in tokenize_d(texts):
    p = morph.parse(token)[0]
    year_pos[p.tag.POS] = year_pos.get(p.tag.POS, 0) + 1
  years_pos_pf[year] = useful_pos(year_pos)

years_pos_pf

{'2012': {'глаголы': 437,
  'деепричастия': 18,
  'наречия': 97,
  'прилагательные': 411,
  'причастия': 25,
  'существительные': 862},
 '2013': {'глаголы': 253,
  'деепричастия': 11,
  'наречия': 92,
  'прилагательные': 212,
  'причастия': 8,
  'существительные': 484},
 '2014': {'глаголы': 373,
  'деепричастия': 13,
  'наречия': 114,
  'прилагательные': 247,
  'причастия': 14,
  'существительные': 633},
 '2015': {'глаголы': 270,
  'деепричастия': 9,
  'наречия': 74,
  'прилагательные': 212,
  'причастия': 19,
  'существительные': 534},
 '2016': {'глаголы': 299,
  'деепричастия': 11,
  'наречия': 78,
  'прилагательные': 207,
  'причастия': 9,
  'существительные': 514},
 '2017': {'глаголы': 488,
  'деепричастия': 7,
  'наречия': 167,
  'прилагательные': 285,
  'причастия': 16,
  'существительные': 685},
 '2019': {'глаголы': 347,
  'деепричастия': 7,
  'наречия': 110,
  'прилагательные': 172,
  'причастия': 10,
  'существительные': 534}}

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame(years_pos_pf)
df
# вот зачем надо делать нормальные функции с самого начала
print(tokenize_ru('хорошо'))
print(tokenize_d('хорошо'))

[]
['хорошо']


In [ ]:
df

,2012,2013,2014,2015,2016,2017,2019
существительные,862,484,633,534,514,685,534
глаголы,437,253,373,270,299,488,347
причастия,25,8,14,19,9,16,10
деепричастия,18,11,13,9,11,7,7
прилагательные,411,212,247,212,207,285,172
наречия,97,92,114,74,78,167,110


# Именованные сущности

In [ ]:
!pip install natasha

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    LOC,
    ORG,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
for album, text in texts_pf.items():
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
      token.lemmatize(morph_vocab)
  doc.parse_syntax(syntax_parser)
  doc.tag_ner(ner_tagger)

  for span in doc.spans:
      span.normalize(morph_vocab)
  # display(doc.spans[:5])
  for span in doc.spans:
      if span.type == PER:
          span.extract_fact(names_extractor)
          print(span.fact)
  #display(doc.spans[:5])
  print({_.normal: _.fact for _ in doc.spans if _.type == PER})

# Наташу обновиили 5 мая, старую версию скачать не удалось. Эта оказалась ещё
# хуже, чем предыдущая, но пользоваться пришлось тем, что есть. Для поиска разных
# именованных сущностей необходимо менять LOC/ORG/PER и просматривать все значения
# т.к. выдает очень много ошибок

{}
None
None
DocFact(slots=[...])
None
None
None
None
DocFact(slots=[...])
None
DocFact(slots=[...])
None
DocFact(slots=[...])
DocFact(slots=[...])
None
None
None
None
None
None
DocFact(slots=[...])
None
None
None
DocFact(slots=[...])
DocFact(slots=[...])
None
None
None
None
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
DocFact(slots=[...])
{'Рвущиеся': None, 'Стенка': None, 'Пело': DocFact(slots=[...]), 'Разрыдаюсь': None, 'Так': None, 'Родина-мать': DocFact(slots=[...]), 'Вместо': None, 'А-а': DocFact(slots=[...]), 'Удары': DocFact(slots=[...]), 'Отмажь': None, 'Тогда': None, 'Благослови': None, 'Плачущей': None, 'Репост': None, 'Глупый': None, 'Иван': DocFact(slots=[...]), 'Как': None, 'Пламенем': None, 'Беззаветно': N

In [ ]:
org = ['Макдональдс', 'Росгвардия', 'Федеральная свора', 'КГБ',
       'Министерство Доброты', 'ФСБ', 'СМИ', 'ГУЛАГ', 'Pussy Riot',
       'ВДВ', 'МВД', 'РПЦ', 'Радио Свободы', 'ДК', 'ЦИК', 'Adidas',
       'Глянцевый рай', 'Орган судебный']
      
loc = ['Лефортово', 'Беслан', 'Дисней Лэнд', 'Донецк', 'Луганск', 'Пенза',
       'Питер', 'Севастополь', 'Крым', 'Крымск', 'Россия', 'Русь', 'Волгоград', 'Сибирь',
        'Болотная Площадь', 'Марс', 'Париж', 'Солнце', 'Астральная поляна',
        'Магадан', 'Земля', 'Фрязино', 'Вудсток']

per = ['Володя', 'Путин', 'Пьеро', 'Березовский', 'Жириновский',
       'Мэри Попинс', 'Иван', 'Наташа', 'Порнофильмы']

org_low = ''
loc_low = ''
per_low = ''

for el in org:
  org_low += el + " "
for el in loc:
  loc_low += el + " "
for el in per:
  per_low += el + " "

org_low = lemmatize(tokenize_ru(org_low))
loc_low = lemmatize(tokenize_ru(loc_low))
per_low = lemmatize(tokenize_ru(per_low))

print(per_low)

years_names_pf = {'2012': '',
             '2013': '',
             '2014': '',
             '2015': '',
             '2016': '',
             '2017': '',
             '2019': ''}

for key, words in year_sorted_lemmas.items():
  names = {'org': [],
           'per': [],
           'loc': []}
  for word in words:
    if word in loc_low:
      names['loc'].append(word)
    elif word in per_low:
      names['per'].append(word)
    elif word in org_low:
      names['org'].append(word)

  years_names_pf[key] = names

['володя', 'путин', 'пьеро', 'березовский', 'жириновский', 'мэри', 'попинс', 'иван', 'наташа', 'порнофильм']


In [ ]:
df = pd.DataFrame(years_names_pf)
df

,2012,2013,2014,2015,2016,2017,2019
org,"[цик, мвд, adidas, орган, судебный, орган, мак...",[свобода],"[pussy, riot, свора, свобода, свобода, свобода...","[свобода, свобода, рай, министерство, вдв]","[рай, свобода, свобода, свобода, фсб, рпц, мин...",[дк],"[макдональдс, росгвардия, федеральный, свора, ..."
per,"[мэри, попинс]","[путин, путин]","[березовский, жириновский, порнофильм, путин]",[],"[путин, пьеро]",[иван],"[володя, володя, володя, володя, володя, волод..."
loc,"[марс, париж, дисней, лэнд, дисней, лэнд, дисн...","[земля, солнце, земля]","[земля, россия, волгоград, крым]","[площадь, солнце, фрязино, земля, вудсток, вуд...","[россия, солнце, солнце, площадь]","[болотный, магадан, сибирь, земля, астральный,...","[земля, солнце, земля, беслан, севастополь, до..."
